In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Starting H2O

import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

In [ ]:
%%time
train = h2o.import_file("../input/tabular-playground-series-jun-2021/train.csv")
test = h2o.import_file("../input/tabular-playground-series-jun-2021/test.csv")
sample_submission = h2o.import_file("/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
sample_submission.head()

In [ ]:
len(test.columns)

In [ ]:
x = test.columns
y = 'target'
train[y] = train[y].asfactor()

In [ ]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=10000, seed=47, max_runtime_secs=30000)
aml.train(x=x, y=y, training_frame=train)

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
# The leader model is stored here
aml.leader

In [ ]:
preds = aml.predict(test)

In [ ]:
preds

In [ ]:
preds = preds.as_data_frame()

In [ ]:
del preds['predict']

In [ ]:
preds.shape

In [ ]:
preds['id'] = test['id'].as_data_frame().values
preds.head()

In [ ]:
preds.to_csv('h2o_automl_submission_1.csv', index=False)
preds.head()